# **Model_Selection**

In [1]:
import numpy as np
import pandas as pd
import gc
import time
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline

warnings.filterwarnings('ignore')

In [2]:
train_activity = pd.read_csv('train_activity.csv')
train_sleep = pd.read_csv('train_sleep.csv')
train_mmse = pd.read_csv('train_mmse.csv')

In [3]:
training_label = pd.read_csv('training_label_activity.csv')

# **train_activity**

train_activity만 사용해서 예측

activity_average_met - 하루간 평균 MET (MET 값이 높을수록 활동 강도가 높다고 볼 수 있다)  
activity_cal_active - 하루간 활동 칼로리  
activity_cal_total - 하루간 총 사용 칼로리  
activity_class_5min - 하루간 5분당 활동 로그  
activity_daily_movement - 매일 움직인 거리  # 사용
activity_day_end - 활동 종료 시간    # 제거
activity_day_start - 활동 시작 시간  # 제거
activity_high - 고강도 활동 시간  
activity_inactive - 비활동 시간  
activity_inactivity_alerts - 비활동 알람 횟수  
activity_low - 저강도 활동 시간  
activity_medium - 중강도 활동 시간  
activity_met_1min - 하루간 1분 당 MET 로그  # 제거
activity_met_min_high - 하루간 고강도 활동 MET  
activity_met_min_inactive - 하루간 비활동 MET  
activity_met_min_low - 하루간 저강도 활동 MET  
activity_met_min_medium - 하루간 중강도 활동 MET  
activity_non_wear - 미착용 시간  
activity_rest - 휴식 시간  
activity_score - 활동 점수  
activity_score_meet_daily_targets - 활동 목표달성 점수  
activity_score_move_every_hour - 매 시간 당 활동유지 점수  
activity_score_recovery_time - 회복시간 점수  
activity_score_stay_active - 활동 유지 점수  
activity_score_training_frequency - 운동 빈도 점수  
activity_score_training_volume - 운동 빈도 점수  
activity_steps - 매일 걸음 수  # mean, max, min, var
activity_total - 활동 총 시간(분)  # mean, max, min, var
CONVERT(activity_class_5min USING utf8) - 하루간 5분당 활동 로그 (UTF-8 변환)  # 제거
CONVERT(activity_met_1min USING utf8) - 하루간 1분 당 MET 로그 (UTF-8 변환)   # 제거

In [4]:
train_activity['activity_non_wear'].describe()

count    9705.000000
mean       44.856981
std        64.379297
min         0.000000
25%         0.000000
50%         2.000000
75%        76.000000
max       300.000000
Name: activity_non_wear, dtype: float64

In [5]:
train_activity[train_activity['activity_non_wear'] > 100]['EMAIL'].value_counts()

EMAIL
nia+119@rowan.kr    51
nia+188@rowan.kr    40
nia+161@rowan.kr    38
nia+043@rowan.kr    36
nia+190@rowan.kr    35
                    ..
nia+421@rowan.kr     2
nia+284@rowan.kr     2
nia+031@rowan.kr     1
nia+336@rowan.kr     1
nia+291@rowan.kr     1
Name: count, Length: 139, dtype: int64

In [6]:
train_activity_groupby = train_activity.groupby('EMAIL')
train_activity_agg = pd.DataFrame()

In [7]:
train_activity_agg['CNT'] = train_activity_groupby['EMAIL'].count()

In [8]:
train_activity_agg

,CNT
EMAIL,
nia+003@rowan.kr,63
nia+005@rowan.kr,113
nia+010@rowan.kr,58
nia+014@rowan.kr,68
nia+015@rowan.kr,35
...,...
nia+451@rowan.kr,38
nia+456@rowan.kr,64
nia+458@rowan.kr,51


In [9]:
train_activity[train_activity['EMAIL'] == 'nia+119@rowan.kr']

,EMAIL,activity_average_met,activity_cal_active,activity_cal_total,activity_class_5min,activity_daily_movement,activity_day_end,activity_day_start,activity_high,activity_inactive,...,activity_score_meet_daily_targets,activity_score_move_every_hour,activity_score_recovery_time,activity_score_stay_active,activity_score_training_frequency,activity_score_training_volume,activity_steps,activity_total,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8)
1558,nia+119@rowan.kr,1.34375,262,2340,...,4460,2020-10-19T03:59:59+09:00,2020-10-18T04:00:00+09:00,0,508,...,78,100,100,82,100,97,5975,283,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/1.1/...
1559,nia+119@rowan.kr,1.37500,333,2384,...,6117,2020-10-20T03:59:59+09:00,2020-10-19T04:00:00+09:00,0,366,...,60,100,100,96,96,96,7686,317,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/1.1/0.9/0.9/0.9/0.9/0.9/1/1.1/0.9/0.9/...
1560,nia+119@rowan.kr,1.37500,310,2371,...,5673,2020-10-21T03:59:59+09:00,2020-10-20T04:00:00+09:00,0,504,...,43,100,100,76,71,93,7721,246,2/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.2/1.4/1.5/2/1.2/1.7/1.2/1.2/1.1/0.9/0.9/0.9/...
1561,nia+119@rowan.kr,1.28125,196,2210,...,3620,2020-10-22T03:59:59+09:00,2020-10-21T04:00:00+09:00,0,559,...,43,100,100,75,71,73,5001,171,1/1/1/1/1/1/1/1/1/1/2/3/2/1/1/1/1/1/1/2/1/1/1/...,0.9/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/1/0.9/0.9/...
1562,nia+119@rowan.kr,1.43750,438,2521,...,8228,2020-10-23T03:59:59+09:00,2020-10-22T04:00:00+09:00,0,391,...,25,100,100,94,40,67,8942,373,2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.7/1/1.2/1.7/1.4/2.7/1.1/0.9/0.9/0.9/0.9/0.9/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1623,nia+119@rowan.kr,1.28125,181,2208,...,3049,2021-01-01T03:59:59+09:00,2020-12-31T04:00:00+09:00,0,577,...,43,100,100,74,1,28,4365,210,1/3/3/3/3/3/2/2/2/2/2/3/3/3/3/1/2/2/2/1/1/2/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/3.4/4.1/1.2/1....
1624,nia+119@rowan.kr,1.18750,174,2126,...,3086,2021-01-02T03:59:59+09:00,2021-01-01T04:00:00+09:00,0,340,...,43,100,100,98,1,30,3980,183,1/1/1/2/2/3/3/3/3/3/3/2/2/2/2/3/2/1/1/1/1/1/1/...,0.9/1.3/0.9/0.9/0.9/1/0.9/0.9/0.9/1.1/0.9/0.9/...
1625,nia+119@rowan.kr,1.43750,425,2500,...,8151,2021-01-03T03:59:59+09:00,2021-01-02T04:00:00+09:00,0,512,...,43,100,100,77,1,46,9515,341,2/1/1/1/1/1/1/1/1/1/1/1/1/1/2/1/1/1/1/1/1/1/1/...,0.9/2.1/0.9/1.3/1.2/0.9/0.9/0.9/0.9/0.9/0.9/0....
1626,nia+119@rowan.kr,1.21875,193,2149,...,3319,2021-01-04T03:59:59+09:00,2021-01-03T04:00:00+09:00,2,451,...,43,100,100,89,1,49,4553,177,1/1/2/2/1/1/1/1/1/1/1/1/1/1/1/1/2/2/2/3/1/1/1/...,1.2/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....


In [10]:
train_activity_agg['activity_average_met_mean'] = train_activity_groupby['activity_average_met'].mean()
train_activity_agg['activity_average_met_var'] = train_activity_groupby['activity_average_met'].var()

train_activity_agg['activity_cal_active_mean'] = train_activity_groupby['activity_cal_active'].mean()
train_activity_agg['activity_cal_active_var'] = train_activity_groupby['activity_cal_active'].var()

train_activity_agg['activity_daily_movement_mean'] = train_activity_groupby['activity_daily_movement'].mean()
train_activity_agg['activity_daily_movement_var'] = train_activity_groupby['activity_daily_movement'].var()

train_activity_agg['activity_low_mean'] = train_activity_groupby['activity_low'].mean()
train_activity_agg['activity_low_var'] = train_activity_groupby['activity_low'].var()

train_activity_agg['activity_inactive_mean'] = train_activity_groupby['activity_inactive'].mean()
train_activity_agg['activity_inactive_var'] = train_activity_groupby['activity_inactive'].var()

train_activity_agg['activity_score_mean'] = train_activity_groupby['activity_score'].mean()
train_activity_agg['activity_score_var'] = train_activity_groupby['activity_score'].var()

In [11]:
train_activity_agg

,CNT,activity_average_met_mean,activity_average_met_var,activity_cal_active_mean,activity_cal_active_var,activity_daily_movement_mean,activity_daily_movement_var,activity_low_mean,activity_low_var,activity_inactive_mean,activity_inactive_var,activity_score_mean,activity_score_var
EMAIL,,,,,,,,,,,,,
nia+003@rowan.kr,63,1.262897,0.009156,202.571429,14455.216590,3659.730159,5.758112e+06,146.428571,3035.797235,615.142857,10467.640553,58.682540,77.994368
nia+005@rowan.kr,113,1.370299,0.004294,217.477876,2696.662453,5084.876106,1.577456e+06,281.300885,3652.855088,574.619469,5720.077118,54.522124,56.876738
nia+010@rowan.kr,58,1.390625,0.005422,259.672414,6939.627647,3877.741379,2.433848e+06,328.827586,6063.583787,632.482759,12616.920750,65.758621,67.309135
nia+014@rowan.kr,68,1.442555,0.010162,401.382353,15424.687445,7250.514706,6.156587e+06,343.852941,6259.440737,534.470588,9370.133450,91.264706,32.854258
nia+015@rowan.kr,35,1.388393,0.006163,271.685714,7150.163025,5053.200000,2.629422e+06,326.485714,7209.963025,484.085714,8083.845378,79.000000,108.176471
...,...,...,...,...,...,...,...,...,...,...,...,...,...
nia+451@rowan.kr,38,1.625822,0.048168,660.394737,26360.785917,12410.868421,1.088854e+07,302.868421,9607.144381,499.973684,11073.107397,90.552632,88.362020
nia+456@rowan.kr,64,1.542480,0.016814,573.781250,31593.125992,10846.750000,1.171415e+07,283.140625,8403.265625,610.515625,16385.555308,87.359375,57.821181
nia+458@rowan.kr,51,1.257353,0.003656,200.549020,4695.412549,3377.098039,1.695171e+06,206.823529,3826.828235,507.117647,9646.145882,61.686275,99.859608


In [12]:
train_activity_agg = train_activity_agg.merge(training_label, left_on='EMAIL', right_on='SAMPLE_EMAIL', how='left')

In [13]:
train_activity_agg_model = train_activity_agg.drop(columns=['SAMPLE_EMAIL'])

In [14]:
train_activity_agg_model

,CNT,activity_average_met_mean,activity_average_met_var,activity_cal_active_mean,activity_cal_active_var,activity_daily_movement_mean,activity_daily_movement_var,activity_low_mean,activity_low_var,activity_inactive_mean,activity_inactive_var,activity_score_mean,activity_score_var,DIAG_NM
0,63,1.262897,0.009156,202.571429,14455.216590,3659.730159,5.758112e+06,146.428571,3035.797235,615.142857,10467.640553,58.682540,77.994368,MCI
1,113,1.370299,0.004294,217.477876,2696.662453,5084.876106,1.577456e+06,281.300885,3652.855088,574.619469,5720.077118,54.522124,56.876738,CN
2,58,1.390625,0.005422,259.672414,6939.627647,3877.741379,2.433848e+06,328.827586,6063.583787,632.482759,12616.920750,65.758621,67.309135,MCI
3,68,1.442555,0.010162,401.382353,15424.687445,7250.514706,6.156587e+06,343.852941,6259.440737,534.470588,9370.133450,91.264706,32.854258,MCI
4,35,1.388393,0.006163,271.685714,7150.163025,5053.200000,2.629422e+06,326.485714,7209.963025,484.085714,8083.845378,79.000000,108.176471,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,38,1.625822,0.048168,660.394737,26360.785917,12410.868421,1.088854e+07,302.868421,9607.144381,499.973684,11073.107397,90.552632,88.362020,CN
137,64,1.542480,0.016814,573.781250,31593.125992,10846.750000,1.171415e+07,283.140625,8403.265625,610.515625,16385.555308,87.359375,57.821181,CN
138,51,1.257353,0.003656,200.549020,4695.412549,3377.098039,1.695171e+06,206.823529,3826.828235,507.117647,9646.145882,61.686275,99.859608,CN
139,41,1.595274,0.024461,654.780488,54600.675610,12247.487805,2.090881e+07,309.951220,4460.697561,473.658537,14099.030488,82.487805,89.006098,MCI


In [15]:
val_activity = pd.read_csv('val_activity.csv')
val_sleep = pd.read_csv('val_sleep.csv')
val_mmse = pd.read_csv('val_mmse.csv')

In [16]:
val_label = pd.read_csv('val_label.csv')

In [17]:
val_activity_groupby = val_activity.groupby('EMAIL')

In [18]:
val_activity_agg = pd.DataFrame()

In [19]:
val_activity_agg['CNT'] = val_activity_groupby['EMAIL'].count()

In [1]:
val_activity_agg

NameError: name 'val_activity_agg' is not defined

In [21]:
val_activity_agg['activity_average_met_mean'] = val_activity_groupby['activity_average_met'].mean()
val_activity_agg['activity_average_met_var'] = val_activity_groupby['activity_average_met'].var()

val_activity_agg['activity_cal_active_mean'] = val_activity_groupby['activity_cal_active'].mean()
val_activity_agg['activity_cal_active_var'] = val_activity_groupby['activity_cal_active'].var()

val_activity_agg['activity_daily_movement_mean'] = val_activity_groupby['activity_daily_movement'].mean()
val_activity_agg['activity_daily_movement_var'] = val_activity_groupby['activity_daily_movement'].var()

val_activity_agg['activity_low_mean'] = val_activity_groupby['activity_low'].mean()
val_activity_agg['activity_low_var'] = val_activity_groupby['activity_low'].var()

val_activity_agg['activity_inactive_mean'] = val_activity_groupby['activity_inactive'].mean()
val_activity_agg['activity_inactive_var'] = val_activity_groupby['activity_inactive'].var()

val_activity_agg['activity_score_mean'] = val_activity_groupby['activity_score'].mean()
val_activity_agg['activity_score_var'] = val_activity_groupby['activity_score'].var()

In [22]:
val_activity_agg = val_activity_agg.merge(val_label, left_on='EMAIL', right_on='SAMPLE_EMAIL', how='left')

In [23]:
val_activity_agg_model = val_activity_agg.drop(columns=['SAMPLE_EMAIL'])

In [24]:
val_activity_agg_model.shape

(33, 14)

In [25]:
train_activity_agg_model.shape

(141, 14)

In [26]:
train_activity_agg_model.drop(columns=['CNT'], inplace=True)
val_activity_agg_model.drop(columns=['CNT'], inplace=True)

In [27]:
train_activity_agg["DIAG_NM"] = train_activity_agg.DIAG_NM.map({"CN":0, "MCI":1, 'Dem':2})
val_activity_agg["DIAG_NM"] = val_activity_agg.DIAG_NM.map({"CN":0, "MCI":1, 'Dem':2})
train_activity_agg_model["DIAG_NM"] = train_activity_agg_model.DIAG_NM.map({"CN":0, "MCI":1, 'Dem':2})
val_activity_agg_model["DIAG_NM"] = val_activity_agg_model.DIAG_NM.map({"CN":0, "MCI":1, 'Dem':2})

In [28]:
train_activity_agg_model["DIAG_NM"]

0      1
1      0
2      1
3      1
4      0
      ..
136    0
137    0
138    0
139    1
140    0
Name: DIAG_NM, Length: 141, dtype: int64

## **1. 로지스틱 회귀** 

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [30]:
# 피처와 타겟 변수 분리
X_train = train_activity_agg_model.drop(columns=["DIAG_NM"])  # 타겟과 EMAIL 제외
y_train = train_activity_agg_model["DIAG_NM"]

X_val = val_activity_agg_model.drop(columns=["DIAG_NM"])  # 타겟과 EMAIL 제외
y_val = val_activity_agg_model["DIAG_NM"]

In [31]:
# 로지스틱 회귀 모델 초기화 및 학습
logistic_model = LogisticRegression(max_iter=1000, class_weight='balanced')  # class_weight='balanced'로 클래스 불균형 처리
logistic_model.fit(X_train, y_train)

# 검증 데이터 예측
y_pred = logistic_model.predict(X_val)

In [32]:
# 정확도 계산
accuracy = accuracy_score(y_val, y_pred)
print("Logistic Regression Accuracy:", accuracy)

# 분류 리포트 출력
print("Classification Report:\n", classification_report(y_val, y_pred))

# 혼동 행렬 출력
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))

Logistic Regression Accuracy: 0.6666666666666666
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.73      0.81        26
           1       0.00      0.00      0.00         4
           2       0.38      1.00      0.55         3

    accuracy                           0.67        33
   macro avg       0.43      0.58      0.45        33
weighted avg       0.75      0.67      0.69        33

Confusion Matrix:
 [[19  4  3]
 [ 2  0  2]
 [ 0  0  3]]


## **로지스틱 회귀 결과해석**

- Class 0 (CN): 19개는 맞게 예측되었지만, 4개는 MCI, 3개는 Dem으로 예측
- Class 1 (MCI): 하나도 못 맞춤
- Class 2 (Dem): 3개 모두 맞게 예측

MCI 예측이 잘 안되고 있음

## **2. 랜덤포레스트** 

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
# 랜덤 포레스트 모델 초기화
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# 모델 학습
rf_model.fit(X_train, y_train)

# 검증 데이터 예측
y_pred_rf = rf_model.predict(X_val)

In [35]:
# 정확도 계산
accuracy_rf = accuracy_score(y_val, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)

# 분류 리포트 출력
print("Classification Report:\n", classification_report(y_val, y_pred_rf))

# 혼동 행렬 출력
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_rf))

Random Forest Accuracy: 0.7272727272727273
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.92      0.84        26
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         3

    accuracy                           0.73        33
   macro avg       0.26      0.31      0.28        33
weighted avg       0.61      0.73      0.66        33

Confusion Matrix:
 [[24  2  0]
 [ 4  0  0]
 [ 3  0  0]]


## **랜덤포레스트 결과해석**

- Class 0 (CN): 24개는 맞게 예측되었지만, 2개는 MCI로 잘못 예측
- Class 1 (MCI): 하나도 못 맞춤
- Class 2 (Dem): 하나도 못 맞춤

Dem이 예측이 안되는건 심각한 문제

## **3. LightGBM** 

In [36]:
import lightgbm as lgb

In [37]:
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

# 수정된 모델 파라미터 설정
params = {
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'is_unbalance': True,  # 클래스 불균형 처리
    'num_leaves': 31,      # num_leaves 설정
    'min_child_samples': 5,  # 기본값보다 낮게 설정
    'max_depth': 6          # 트리의 최대 깊이를 제한
}

# 모델 재학습
lgb_model = lgb.train(params, train_data, num_boost_round=100, valid_sets=[val_data], early_stopping_rounds=10)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000095 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 576
[LightGBM] [Info] Number of data points in the train set: 141, number of used features: 12
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score -0.506109
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -2.751535
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM

In [38]:
# 검증 데이터 예측
y_pred_lgb = lgb_model.predict(X_val)
y_pred_lgb_classes = [np.argmax(row) for row in y_pred_lgb]  # 확률 값 중 가장 큰 값을 클래스로 변환

# 정확도 계산
accuracy_lgb = accuracy_score(y_val, y_pred_lgb_classes)
print("LightGBM Accuracy:", accuracy_lgb)

# 분류 리포트 출력
print("Classification Report:\n", classification_report(y_val, y_pred_lgb_classes))

# 혼동 행렬 출력
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_lgb_classes))

LightGBM Accuracy: 0.696969696969697
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.88      0.82        26
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         3

    accuracy                           0.70        33
   macro avg       0.26      0.29      0.27        33
weighted avg       0.60      0.70      0.65        33

Confusion Matrix:
 [[23  3  0]
 [ 4  0  0]
 [ 3  0  0]]


# **train_mmse의 total을 추가해서 학습**

In [41]:
# train_mmse의 'TOTAL' 피처를 SAMPLE_EMAIL 기준으로 병합
train_activity_agg = train_activity_agg.merge(train_mmse[['SAMPLE_EMAIL', 'TOTAL']], on='SAMPLE_EMAIL', how='left')

# val_mmse의 'TOTAL' 피처를 SAMPLE_EMAIL 기준으로 병합
val_activity_agg = val_activity_agg.merge(val_mmse[['SAMPLE_EMAIL', 'TOTAL']], on='SAMPLE_EMAIL', how='left')

# 결과 확인
train_activity_agg.head()

,CNT,activity_average_met_mean,activity_average_met_var,activity_cal_active_mean,activity_cal_active_var,activity_daily_movement_mean,activity_daily_movement_var,activity_low_mean,activity_low_var,activity_inactive_mean,activity_inactive_var,activity_score_mean,activity_score_var,SAMPLE_EMAIL,DIAG_NM,TOTAL_x,TOTAL_y
0,63,1.262897,0.009156,202.571429,14455.216590,3659.730159,5.758112e+06,146.428571,3035.797235,615.142857,10467.640553,58.682540,77.994368,nia+003@rowan.kr,1,18,18
1,113,1.370299,0.004294,217.477876,2696.662453,5084.876106,1.577456e+06,281.300885,3652.855088,574.619469,5720.077118,54.522124,56.876738,nia+005@rowan.kr,0,28,28
2,58,1.390625,0.005422,259.672414,6939.627647,3877.741379,2.433848e+06,328.827586,6063.583787,632.482759,12616.920750,65.758621,67.309135,nia+010@rowan.kr,1,28,28
3,68,1.442555,0.010162,401.382353,15424.687445,7250.514706,6.156587e+06,343.852941,6259.440737,534.470588,9370.133450,91.264706,32.854258,nia+014@rowan.kr,1,28,28
4,35,1.388393,0.006163,271.685714,7150.163025,5053.200000,2.629422e+06,326.485714,7209.963025,484.085714,8083.845378,79.000000,108.176471,nia+015@rowan.kr,0,26,26


In [42]:
train_activity_agg.drop(columns=['TOTAL_y'], inplace=True)
val_activity_agg.drop(columns=['TOTAL_y'], inplace=True)

In [43]:
train_activity_agg

,CNT,activity_average_met_mean,activity_average_met_var,activity_cal_active_mean,activity_cal_active_var,activity_daily_movement_mean,activity_daily_movement_var,activity_low_mean,activity_low_var,activity_inactive_mean,activity_inactive_var,activity_score_mean,activity_score_var,SAMPLE_EMAIL,DIAG_NM,TOTAL_x
0,63,1.262897,0.009156,202.571429,14455.216590,3659.730159,5.758112e+06,146.428571,3035.797235,615.142857,10467.640553,58.682540,77.994368,nia+003@rowan.kr,1,18
1,113,1.370299,0.004294,217.477876,2696.662453,5084.876106,1.577456e+06,281.300885,3652.855088,574.619469,5720.077118,54.522124,56.876738,nia+005@rowan.kr,0,28
2,58,1.390625,0.005422,259.672414,6939.627647,3877.741379,2.433848e+06,328.827586,6063.583787,632.482759,12616.920750,65.758621,67.309135,nia+010@rowan.kr,1,28
3,68,1.442555,0.010162,401.382353,15424.687445,7250.514706,6.156587e+06,343.852941,6259.440737,534.470588,9370.133450,91.264706,32.854258,nia+014@rowan.kr,1,28
4,35,1.388393,0.006163,271.685714,7150.163025,5053.200000,2.629422e+06,326.485714,7209.963025,484.085714,8083.845378,79.000000,108.176471,nia+015@rowan.kr,0,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,38,1.625822,0.048168,660.394737,26360.785917,12410.868421,1.088854e+07,302.868421,9607.144381,499.973684,11073.107397,90.552632,88.362020,nia+451@rowan.kr,0,30
137,64,1.542480,0.016814,573.781250,31593.125992,10846.750000,1.171415e+07,283.140625,8403.265625,610.515625,16385.555308,87.359375,57.821181,nia+456@rowan.kr,0,27
138,51,1.257353,0.003656,200.549020,4695.412549,3377.098039,1.695171e+06,206.823529,3826.828235,507.117647,9646.145882,61.686275,99.859608,nia+458@rowan.kr,0,28
139,41,1.595274,0.024461,654.780488,54600.675610,12247.487805,2.090881e+07,309.951220,4460.697561,473.658537,14099.030488,82.487805,89.006098,nia+462@rowan.kr,1,25


In [44]:
val_activity_agg

,CNT,activity_average_met_mean,activity_average_met_var,activity_cal_active_mean,activity_cal_active_var,activity_daily_movement_mean,activity_daily_movement_var,activity_low_mean,activity_low_var,activity_inactive_mean,activity_inactive_var,activity_score_mean,activity_score_var,SAMPLE_EMAIL,DIAG_NM,TOTAL_x
0,122,1.478484,0.023939,493.147541,50165.664002,8769.811475,1.817404e+07,314.918033,7883.232895,481.016393,10012.280721,91.590164,24.343043,nia+013@rowan.kr,1,30
1,62,1.456653,0.005326,496.919355,13977.944209,9172.741935,5.166737e+06,185.645161,6526.560550,506.967742,4895.507139,90.935484,54.421999,nia+022@rowan.kr,1,29
2,61,1.334016,0.005877,276.704918,9442.811475,5059.049180,3.569895e+06,189.180328,4316.583607,641.704918,15289.844809,67.491803,137.254098,nia+027@rowan.kr,0,27
3,107,1.333820,0.014549,262.149533,35081.618938,5389.542056,1.577509e+07,173.971963,1769.914301,622.177570,8726.373832,64.859813,298.857521,nia+045@rowan.kr,2,0
4,70,1.373214,0.016160,339.000000,14201.884058,6246.128571,5.681935e+06,303.271429,7729.823810,528.900000,7115.337681,77.728571,99.736853,nia+064@rowan.kr,0,29
5,93,1.474798,0.011724,441.892473,24192.336138,8292.021505,1.034928e+07,362.440860,6865.227443,509.129032,9514.417952,88.161290,50.093268,nia+084@rowan.kr,0,29
6,78,1.379407,0.007298,350.576923,12351.026474,6500.769231,4.817066e+06,231.948718,6166.075258,551.038462,8669.180320,80.846154,85.716284,nia+100@rowan.kr,0,28
7,40,1.095312,0.002051,64.475000,2109.794231,1094.150000,8.700337e+05,60.650000,1102.746154,385.625000,15401.727564,54.250000,109.525641,nia+112@rowan.kr,2,11
8,60,1.581250,0.009243,682.716667,21433.799718,11854.866667,6.730211e+06,153.600000,1533.091525,666.033333,6194.337853,73.000000,86.915254,nia+128@rowan.kr,1,27
9,69,1.523551,0.015723,495.724638,35384.526002,9387.000000,1.536789e+07,412.420290,13968.158994,477.681159,12437.367434,75.173913,142.851662,nia+163@rowan.kr,0,27


In [45]:
train_activity_agg.drop(columns=['CNT'], inplace=True)
val_activity_agg.drop(columns=['CNT'], inplace=True)

In [47]:
# 피처와 타겟 변수 분리
X_train = train_activity_agg.drop(columns=["DIAG_NM", 'SAMPLE_EMAIL'])  # 타겟과 EMAIL 제외
y_train = train_activity_agg["DIAG_NM"]

X_val = val_activity_agg.drop(columns=["DIAG_NM", 'SAMPLE_EMAIL'])  # 타겟과 EMAIL 제외
y_val = val_activity_agg["DIAG_NM"]

In [48]:
# 로지스틱 회귀 모델 초기화 및 학습
logistic_model = LogisticRegression(max_iter=1000, class_weight='balanced')  # class_weight='balanced'로 클래스 불균형 처리
logistic_model.fit(X_train, y_train)

# 검증 데이터 예측
y_pred = logistic_model.predict(X_val)

In [49]:
# 정확도 계산
accuracy = accuracy_score(y_val, y_pred)
print("Logistic Regression Accuracy:", accuracy)

# 분류 리포트 출력
print("Classification Report:\n", classification_report(y_val, y_pred))

# 혼동 행렬 출력
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))

Logistic Regression Accuracy: 0.6060606060606061
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.65      0.76        26
           1       0.00      0.00      0.00         4
           2       0.38      1.00      0.55         3

    accuracy                           0.61        33
   macro avg       0.42      0.55      0.43        33
weighted avg       0.74      0.61      0.64        33

Confusion Matrix:
 [[17  6  3]
 [ 2  0  2]
 [ 0  0  3]]


In [50]:
# 랜덤 포레스트 모델 초기화
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# 모델 학습
rf_model.fit(X_train, y_train)

# 검증 데이터 예측
y_pred_rf = rf_model.predict(X_val)

In [51]:
# 정확도 계산
accuracy_rf = accuracy_score(y_val, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)

# 분류 리포트 출력
print("Classification Report:\n", classification_report(y_val, y_pred_rf))

# 혼동 행렬 출력
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_rf))

Random Forest Accuracy: 0.7878787878787878
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.92      0.89        26
           1       0.00      0.00      0.00         4
           2       1.00      0.67      0.80         3

    accuracy                           0.79        33
   macro avg       0.62      0.53      0.56        33
weighted avg       0.77      0.79      0.77        33

Confusion Matrix:
 [[24  2  0]
 [ 4  0  0]
 [ 0  1  2]]


In [52]:
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

# 수정된 모델 파라미터 설정
params = {
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'is_unbalance': True,  # 클래스 불균형 처리
    'num_leaves': 31,      # num_leaves 설정
    'min_child_samples': 5,  # 기본값보다 낮게 설정
    'max_depth': 6          # 트리의 최대 깊이를 제한
}

# 모델 재학습
lgb_model = lgb.train(params, train_data, num_boost_round=100, valid_sets=[val_data], early_stopping_rounds=10)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000149 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 589
[LightGBM] [Info] Number of data points in the train set: 141, number of used features: 13
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score -0.506109
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -2.751535
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM

In [53]:
# 검증 데이터 예측
y_pred_lgb = lgb_model.predict(X_val)
y_pred_lgb_classes = [np.argmax(row) for row in y_pred_lgb]  # 확률 값 중 가장 큰 값을 클래스로 변환

# 정확도 계산
accuracy_lgb = accuracy_score(y_val, y_pred_lgb_classes)
print("LightGBM Accuracy:", accuracy_lgb)

# 분류 리포트 출력
print("Classification Report:\n", classification_report(y_val, y_pred_lgb_classes))

# 혼동 행렬 출력
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_lgb_classes))

LightGBM Accuracy: 0.7575757575757576
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.88      0.87        26
           1       0.00      0.00      0.00         4
           2       1.00      0.67      0.80         3

    accuracy                           0.76        33
   macro avg       0.62      0.52      0.56        33
weighted avg       0.76      0.76      0.76        33

Confusion Matrix:
 [[23  3  0]
 [ 4  0  0]
 [ 0  1  2]]


### **해석**

1. 치매환자 데이터가 너무 적음. 데이터를 증강할 필요성이 있어보임 (날짜를 나누거나, SMOTE를 쓰거나)

2. MCI를 구분하는건 힘들어보이니 치매-비치매 이진분류로 접근하는게 좋아보임

3. 0 1 2로 맵핑하는게 유효하지 않았을수도.